# Pushing your own model to hub

In [ ]:
# authenicating to hugging face hub api
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install accelerate -U


In [ ]:
!pip install datasets

In [ ]:
# Loading datasets
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("glue","cola")                                           # loading cola datasets from dataset

In [ ]:
raw_datasets

In [ ]:
from transformers import AutoTokenizer
checkpoint = "bert-base-cased"                                              # using pretrained model checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint)                       # loading tokenizer from model checkpoint

In [ ]:
def preprocess_function(datasets):
  return tokenizer(datasets["sentence"], truncation=True)                             # function to tokenize datasets

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)              # tokenizing raw datasets in batch


In [ ]:
# loading pretrained model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)                                    # using pretrained model with its checkpoint

In [ ]:
# creating evaluation metrics
from datasets import load_metric
import numpy as np

metric = load_metric("glue", "cola")                                                                      # loading metric for cola datasets

# funciton to calculate evaluation metrics
def compute_metrics(eval_preds):
  predicitions, labels = eval_preds                                                                      # logits and label
  predictions = np.argmax(predicitions, axis=-1)                                                         # getting max logits
  return metric.compute(predictions=predictions, references=labels)                                          # calculating metrics form predictions and refrence label

<ipython-input-12-8622ba85ac3a>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "cola")                                                                      # loading metric for cola datasets
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# creating training arguments
from transformers import TrainingArguments
training_args = TrainingArguments(
    "bert-fined-tuned-cola",                                        # name of  output directory
    evaluation_strategy="epoch",                                    # evaluating performance at every epoch
    learning_rate= 2e-5,                                            # Selecting proper learning rate
    num_train_epochs=3,                                             # specifying number of epochs to train
    weight_decay=0.001,                                             # setting up regularization
    save_strategy="epoch",                                          # model will be saved at every epoch
     push_to_hub=True                                               # model will be push to hugging face hub
)

In [ ]:
# preparing for model trainig
from transformers import Trainer
trainer = Trainer(
    model,                                                         # model used
    training_args,                                                 # all the parameters for model training
    train_dataset = tokenized_datasets["train"],                   # training datasets
    eval_dataset = tokenized_datasets["validation"],               # validation datasets
    compute_metrics=compute_metrics,                               # calcualting metrics
    tokenizer=tokenizer,                                           # tokenizer used

)
trainer.train()

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.453000,0.437257,0.518867
2,0.312400,0.677438,0.572681
3,0.196600,0.784551,0.562623


TrainOutput(global_step=3207, training_loss=0.3367439331873057, metrics={'train_runtime': 318.6839, 'train_samples_per_second': 80.497, 'train_steps_per_second': 10.063, 'total_flos': 243897267764580.0, 'train_loss': 0.3367439331873057, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub("End of traininig")

CommitInfo(commit_url='https://huggingface.co/Utshav/bert-fined-tuned-cola/commit/99de36858ad3b65278d03d589eafbb9f45f38eab', commit_message='End of traininig', commit_description='', oid='99de36858ad3b65278d03d589eafbb9f45f38eab', pr_url=None, pr_revision=None, pr_num=None)